<a href="https://colab.research.google.com/github/MatteoGhizzi/MLPNS2021/blob/main/Lezione6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import scipy as sp
from scipy import stats

##L'obbiettivo è generare set di valori di dimensioni differinti (100, 1000, 10000) da una distribuzione normale e confrontarli tramite il K-S test con la normale standard.

##Il K-S test verrà poi utilizzato per confrontare varie distribuzioni con la distribuzione normale standard

In [9]:
X = sp.stats.norm.rvs(size=100)
Y = sp.stats.norm.rvs(size=1000)
Z = sp.stats.norm.rvs(size=10000)
df = 1
CHI = sp.stats.chi2.rvs(df, size=1000)

In [10]:
sp.stats.kstest(X, 'norm')

KstestResult(statistic=0.06782400580542214, pvalue=0.7626232947611578)

In [11]:
sp.stats.kstest(Y, 'norm')

KstestResult(statistic=0.020536822604793503, pvalue=0.7928931457709346)

In [12]:
sp.stats.kstest(Z, 'norm')

KstestResult(statistic=0.010530372837556146, pvalue=0.21742128531776203)

In [13]:
sp.stats.kstest(CHI, 'norm')


KstestResult(statistic=0.5000009011624119, pvalue=1.0623315523337076e-231)

#statistic is the maximum distance between the 2 comulative distribution
# p value must be << than my thresh
# ripsonde alla domanda : provengono dalla stessa distribuzione ??

In [15]:
 sp.stats.ks_2samp(X,Y)

Ks_2sampResult(statistic=0.065, pvalue=0.8177791664689186)

## PARTE 2 

#riproduzione articolo earthquakes

In [16]:
#all the imports I needed
from __future__ import print_function, division
import pandas as pd
import pylab as pl
import scipy as sp
from scipy import stats
%pylab inline
pl.style.use("https://raw.githubusercontent.com/fedhere/DSPS/master/fbb.mplstyle")

Populating the interactive namespace from numpy and matplotlib


In [19]:
# IMPORTIAMO I DATI dal raw file presente su Github
data = pd.read_csv('https://raw.githubusercontent.com/fedhere/MLPNS2021/main/NHRT/earthquakes.csv',sep=' ')

In [18]:
data.describe


<bound method NDFrame.describe of        #YYY/MM/DD  HH:mm:SS.ss  ET  GT  ... Unnamed: 30  Unnamed: 31 NPH  NGRM
0      1984/01/01  01:30:57.52  eq NaN  ...         NaN          NaN NaN   NaN
1      1984/01/01  05:31:34.16  eq NaN  ...         NaN          NaN NaN   NaN
2      1984/01/01  06:59:05.14  eq NaN  ...         NaN          NaN NaN   NaN
3      1984/01/01  11:10:25.29  eq NaN  ...         NaN          NaN NaN   NaN
4      1984/01/01  19:38:19.79  eq NaN  ...         NaN          NaN NaN   NaN
...           ...          ...  ..  ..  ...         ...          ...  ..   ...
70793  2002/12/29  19:37:24.56  eq NaN  ...         NaN          NaN NaN   NaN
70794  2002/12/29  20:17:33.40  eq NaN  ...         NaN          NaN NaN   NaN
70795  2002/12/29  20:21:28.73  eq NaN  ...         NaN          NaN NaN   NaN
70796  2002/12/30  21:51:52.55  eq NaN  ...         NaN          NaN NaN   NaN
70797  2002/12/30  23:38:49.79  eq NaN  ...         NaN          NaN NaN   NaN

[70798 rows x 34 

In [22]:
# cambiamo nome ai dati e visualizziamo quelli che ci interessano
eqdata = data.rename({'#YYY/MM/DD':'date','HH:mm:SS.ss':'time','Unnamed: 5':'mag'},axis=1)[['date','time','mag']]
eqdata.head()


,date,time,mag
0,1984/01/01,01:30:57.52,2.02
1,1984/01/01,05:31:34.16,2.12
2,1984/01/01,06:59:05.14,2.43
3,1984/01/01,11:10:25.29,2.07
4,1984/01/01,19:38:19.79,2.13


In [23]:
# devo convertire in uno tipo di file più utilizzabile  e unire queste due colonne in una unica
eqdata.time.dtype, eqdata.date.dtype


(dtype('O'), dtype('O'))

In [24]:
#delete
#fixing time which ends with 60.00 sec
for i in range(len(eqdata)):            # faccio passare tutte le righe della colonna time
    if eqdata.iloc[i].time.endswith('60.00'):   # se per ogni riga iloc(i) la colonna time di data finisce per 60.00
        print(eqdata.iloc[i].time[:6] + "59.99"). # stampa dal sesto numero compreso e poi stampa a fianco 59.99
        eqdata["time"][i] = eqdata.iloc[i].time[:6] + "59.99".  # qui eseguo la sostituzione
        print (eqdata.iloc[i].time)


04:48:59.99
04:48:59.99
14:19:59.99
14:19:59.99


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


21:48:59.99
21:48:59.99
15:34:59.99
15:34:59.99
06:41:59.99
06:41:59.99


In [27]:
# questo comando crea una nuova colonna  che è l'unione della colonna data e 
#time e le converte in time per pandas
#usiamo al posto del for loop classico un comando compreso nella lista
eqdata['datetime'] = pd.to_datetime([eqdata.iloc[i].date + " " + 
                            eqdata.iloc[i].time for i in range(len(eqdata))])

In [28]:
eqdata.head()

,date,time,mag,datetime
0,1984/01/01,01:30:57.52,2.02,1984-01-01 01:30:57.520
1,1984/01/01,05:31:34.16,2.12,1984-01-01 05:31:34.160
2,1984/01/01,06:59:05.14,2.43,1984-01-01 06:59:05.140
3,1984/01/01,11:10:25.29,2.07,1984-01-01 11:10:25.290
4,1984/01/01,19:38:19.79,2.13,1984-01-01 19:38:19.790


#a me servirebbero gli intervalli tmeporali

In [30]:
# first I create the timetamps pairs for the beginning and end of the valid periods. 
# I do it with a for loop that appends pairs of timestamps to a list that I create at the beginning
# create empty container
ss = []

# loop through all year pairs as year and fraction of year
for t in [(1984, 1986.5), 
          (1990.3, 1992.1), 
          (1994.6, 1995.6),
          (1996.1,1996.5), 
          (1997,1997.6), 
          (1997.75,1998.15), 
          (1998.25,1999.35), 
          (2000.55,2000.8),
          (2000.9 ,2001.25), 
          (2001.6 ,2002), 
          (2002.5 ,2003)]:
    
    # for each pair I convert the year to datetime (easy)
    # then I convert the fraction of year and convert it to days and convert that to a delta-time
    # finally I add that deltatime to the year datetime. 
    ss.append((pd.to_datetime(int(t[0]), format="%Y") 
        + pd.Timedelta('%.f days'%((t[0] - int(t[0])) * 365.25)),  # prendo gli elementi e faccio la differenza
               pd.to_datetime(int(t[1]), format="%Y")   # ottengo un valore in frazione di anni allora molptipllico per 365.25 per ottenere il valore in giorni ecc
        + pd.Timedelta('%.f days'%((t[1] - int(t[1])) * 365.25))))

In [31]:
ss

[(Timestamp('1984-01-01 00:00:00'), Timestamp('1986-07-03 00:00:00')),
 (Timestamp('1990-04-21 00:00:00'), Timestamp('1992-02-07 00:00:00')),
 (Timestamp('1994-08-08 00:00:00'), Timestamp('1995-08-08 00:00:00')),
 (Timestamp('1996-02-07 00:00:00'), Timestamp('1996-07-02 00:00:00')),
 (Timestamp('1997-01-01 00:00:00'), Timestamp('1997-08-08 00:00:00')),
 (Timestamp('1997-10-02 00:00:00'), Timestamp('1998-02-25 00:00:00')),
 (Timestamp('1998-04-02 00:00:00'), Timestamp('1999-05-09 00:00:00')),
 (Timestamp('2000-07-20 00:00:00'), Timestamp('2000-10-19 00:00:00')),
 (Timestamp('2000-11-25 00:00:00'), Timestamp('2001-04-02 00:00:00')),
 (Timestamp('2001-08-08 00:00:00'), Timestamp('2002-01-01 00:00:00')),
 (Timestamp('2002-07-03 00:00:00'), Timestamp('2003-01-01 00:00:00'))]

In [32]:
# l'articolo mi dice di prendere solo terremoti per mag > 2
# devo fare un BROADCASTING  IMPORTANTE
print ("the {} timestamp pairs that are the boundaries of good data periods are:".format(len(ss)))

the 11 timestamp pairs that are the boundaries of good data periods are:


In [33]:
# data between start and end of first time period
gooddates = (((eqdata.datetime >= ss[0][0]) * (eqdata.datetime <= ss[0][1])) +\
        # data between start and end of second time period
        ((eqdata.datetime >=ss [1][0]) * (eqdata.datetime <= ss[1][1])) +\
        ((eqdata.datetime >=ss [2][0]) * (eqdata.datetime <= ss[2][1])) +\
        ((eqdata.datetime >=ss [3][0]) * (eqdata.datetime <= ss[3][1])) +\
        ((eqdata.datetime >=ss [4][0]) * (eqdata.datetime <= ss[4][1])) +\
        ((eqdata.datetime >=ss [5][0]) * (eqdata.datetime <= ss[5][1])) +\
        ((eqdata.datetime >=ss [6][0]) * (eqdata.datetime <= ss[6][1])) +\
        ((eqdata.datetime >=ss [7][0]) * (eqdata.datetime <= ss[7][1])) +\
        ((eqdata.datetime >=ss [8][0]) * (eqdata.datetime <= ss[8][1])) +\
        ((eqdata.datetime >=ss [9][0]) * (eqdata.datetime <= ss[9][1])) +\
        # data between start and end of last time period
        ((eqdata.datetime >=ss [10][0]) * (eqdata.datetime <= ss[10][1])\
        ).values)

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [34]:
# broadcasting to create a dataframe with the good dates only
eqGoodData = eqdata[gooddates]
eqGoodData.shape


(18852, 4)